In [1]:
## 1. Importing Libraries for usage of the project.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format='retina' # display figures in retina quality
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

from sklearn.preprocessing import LabelEncoder

In [2]:
## 2. Importing our dataset and drop error NA values.

df = pd.read_csv('BC_driver_license_test.csv')

/var/folders/nb/6qcxnwwn63qflk99ycftkdcr0000gn/T/ipykernel_11413/1908921218.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('BC_driver_license_test.csv')


In [3]:
df = df.dropna()
df.shape

(1417534, 12)

In [4]:
## 3.Datawashing, clean the unnecessary details of strs.


In [5]:
driver_exam_breakdown_indicator = []
for i in range(len(df['driver_exam_breakdown'])):
    driver_exam_breakdown_indicator.append(df['driver_exam_breakdown'][i][6])
    
df['driver_exam_breakdown'] = driver_exam_breakdown_indicator

In [6]:
df

,test_id,driver_exam_breakdown,year,exam_month,exam_reason,exam_result,exam_type,gender,office_municiplaity,region,exam_class,exam_date
0,0,6,2017,May,Re-test,Failed,Motorcycle Skills,Male,SURREY,Lower Mainland,6,2017-05-09
1,1,6,2017,May,Re-test,Failed,Motorcycle Skills,Male,SURREY,Lower Mainland,6,2017-05-09
2,2,6,2017,May,Re-test,Passed,Motorcycle Skills,Female,PORT HARDY,Vancouver Island,6,2017-05-09
3,3,6,2017,May,Re-test,Passed,Motorcycle Skills,Male,SURREY,Lower Mainland,6,2017-05-09
4,4,6,2017,May,Re-test,Passed,Motorcycle Skills,Male,SURREY,Lower Mainland,6,2017-05-09
...,...,...,...,...,...,...,...,...,...,...,...,...
1417529,1417529,5,2018,June,Class Change,Passed,Road,Female,NANAIMO DLO,Vancouver Island,5,2018-06-20
1417530,1417530,5,2018,June,Class Change,Passed,Road,Female,NANAIMO DLO,Vancouver Island,5,2018-06-20
1417531,1417531,5,2018,June,Class Change,Passed,Road,Female,NANAIMO DLO,Vancouver Island,5,2018-06-25
1417532,1417532,5,2018,June,Class Change,Passed,Road,Female,NANAIMO DLO,Vancouver Island,5,2018-06-25


In [7]:
# 4.Costumize our label encoder, since using label encoder.applyall will mess up infos with weird values.
##  Use Supervised learning method le. -> label encoder to automate binary transform our data.
##  Since we have our label encoder ----> le. we can use all methods from le.

## encode exam_month by specifying the encoder.class ---> else the May will be displayed as 8.
# Define a custom mapping of month names to numerical labels
month_map = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5,
             'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'Novemember': 11, 'December':12}

# Create a LabelEncoder object and set its mapping to the custom mapping
le = LabelEncoder()
le.fit(df['exam_month'].str.strip().map(month_map))

# Transform the exam_month column into numerical labels
df['exam_month_encoded'] = le.transform(df['exam_month'].str.strip().map(month_map)) + 1
df['exam_month'] = df['exam_month_encoded']

In [8]:
df

,test_id,driver_exam_breakdown,year,exam_month,exam_reason,exam_result,exam_type,gender,office_municiplaity,region,exam_class,exam_date,exam_month_encoded
0,0,6,2017,5,Re-test,Failed,Motorcycle Skills,Male,SURREY,Lower Mainland,6,2017-05-09,5
1,1,6,2017,5,Re-test,Failed,Motorcycle Skills,Male,SURREY,Lower Mainland,6,2017-05-09,5
2,2,6,2017,5,Re-test,Passed,Motorcycle Skills,Female,PORT HARDY,Vancouver Island,6,2017-05-09,5
3,3,6,2017,5,Re-test,Passed,Motorcycle Skills,Male,SURREY,Lower Mainland,6,2017-05-09,5
4,4,6,2017,5,Re-test,Passed,Motorcycle Skills,Male,SURREY,Lower Mainland,6,2017-05-09,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417529,1417529,5,2018,6,Class Change,Passed,Road,Female,NANAIMO DLO,Vancouver Island,5,2018-06-20,6
1417530,1417530,5,2018,6,Class Change,Passed,Road,Female,NANAIMO DLO,Vancouver Island,5,2018-06-20,6
1417531,1417531,5,2018,6,Class Change,Passed,Road,Female,NANAIMO DLO,Vancouver Island,5,2018-06-25,6
1417532,1417532,5,2018,6,Class Change,Passed,Road,Female,NANAIMO DLO,Vancouver Island,5,2018-06-25,6


In [9]:
#-------------------------------------------------------------------------------------
# Now We've established cleaning for classes and month, let's apply encoding to all other components that
# does not involve any numerical relationship with the string.

# first train le to fit exam_reason.
le.fit(df['exam_reason'])
df['exam_reason_encoded'] = le.transform(df['exam_reason'])



In [10]:
# Verify each corresponding components.
print(df['exam_reason'].unique())

['Re-test' 'Upgrade' 'Re-exam' 'Class Change' 'Original'
 'Restriction Change' 'Information not available' 'Downgrade'
 'New Licence']


In [11]:
print(df['exam_reason_encoded'].unique())

[6 8 5 0 4 7 2 1 3]


##### Where we could've categorize them in this tab.
#####

## Exam_Reason:
    0 - Class Change              1 - Downgrade
    2 - Information not available 3 - New Licence
    4 - Original                  5 - Re-exam
    6 - Re-test                   7 - Restriction Change
    8 - Upgrade
    



In [12]:
## train and fit exam result
## Which is a simple binary, Pass = 1, Fail = 0.
le.fit(df['exam_result'])
df['exam_result_encoded'] = le.transform(df['exam_result'])

In [13]:
## train and fit exam_type
## Which is a binary as well, Motorcycle = 0, Road = 1.
print(df['exam_type'].unique())
le.fit(df['exam_type'])
df['exam_type_encoded'] = le.transform(df['exam_type'])

['Motorcycle Skills' 'Road']


In [14]:
## train and fit gender.
## Where there are 3 values, Male = 2 , Female = 0, INA =1.
print(df['gender'].unique())
le.fit(df['gender'])
df['gender_encoded'] = le.transform(df['gender'])

['Male' 'Female' 'Information not available']


In [15]:
## train and fit office_municiplaity
## Since location has multiple locations, we have to display them using tables.
le.fit(df['office_municiplaity'])
df['office_municiplaity_encoded'] = le.transform(df['office_municiplaity'])

In [16]:
cities = df['office_municiplaity'].unique()

In [17]:
reps = df['office_municiplaity_encoded'].unique()
display_df = pd.DataFrame({
    'office_municiplaity': cities,
    'office_municiplaity_encoded': reps
})
## Since the cities were too much, I've created a dataframe to display it externally.
display_df

,office_municiplaity,office_municiplaity_encoded
0,SURREY,100
1,PORT HARDY,74
2,COURTENAY,20
3,NANAIMO DLO,60
4,PORT COQUITLAM,73
...,...,...
110,CHASE,14
111,LAKE COWICHAN,48
112,RICHMOND MINORU,86
113,SALMO,88


In [18]:
## train and fit region
## Since region has multiple locations, we have to display them using tables.
le.fit(df['region'])
df['region_encoded'] = le.transform(df['region'])

In [19]:
## Aligning
## Lower Mainland: 0   Vancouver Island: 4
## North Central: 1    Southern Interior: 2
## Unknown: 3
df['region'].unique()

array(['Lower Mainland', 'Vancouver Island', 'North Central',
       'Southern Interior', 'Unknown'], dtype=object)

In [20]:
df['region_encoded'].unique()

array([0, 4, 1, 2, 3])

In [21]:
df

,test_id,driver_exam_breakdown,year,exam_month,exam_reason,exam_result,exam_type,gender,office_municiplaity,region,exam_class,exam_date,exam_month_encoded,exam_reason_encoded,exam_result_encoded,exam_type_encoded,gender_encoded,office_municiplaity_encoded,region_encoded
0,0,6,2017,5,Re-test,Failed,Motorcycle Skills,Male,SURREY,Lower Mainland,6,2017-05-09,5,6,0,0,2,100,0
1,1,6,2017,5,Re-test,Failed,Motorcycle Skills,Male,SURREY,Lower Mainland,6,2017-05-09,5,6,0,0,2,100,0
2,2,6,2017,5,Re-test,Passed,Motorcycle Skills,Female,PORT HARDY,Vancouver Island,6,2017-05-09,5,6,1,0,0,74,4
3,3,6,2017,5,Re-test,Passed,Motorcycle Skills,Male,SURREY,Lower Mainland,6,2017-05-09,5,6,1,0,2,100,0
4,4,6,2017,5,Re-test,Passed,Motorcycle Skills,Male,SURREY,Lower Mainland,6,2017-05-09,5,6,1,0,2,100,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417529,1417529,5,2018,6,Class Change,Passed,Road,Female,NANAIMO DLO,Vancouver Island,5,2018-06-20,6,0,1,1,0,60,4
1417530,1417530,5,2018,6,Class Change,Passed,Road,Female,NANAIMO DLO,Vancouver Island,5,2018-06-20,6,0,1,1,0,60,4
1417531,1417531,5,2018,6,Class Change,Passed,Road,Female,NANAIMO DLO,Vancouver Island,5,2018-06-25,6,0,1,1,0,60,4
1417532,1417532,5,2018,6,Class Change,Passed,Road,Female,NANAIMO DLO,Vancouver Island,5,2018-06-25,6,0,1,1,0,60,4


In [22]:
##5. After we've encoded all columns that we need, now we've can create a new column only contains the encoded codes,
###  and count for the option that has the greatest correlation with pass for exam.

In [23]:
edf = df.loc[:, ['test_id', 'exam_class', 'year', 'exam_month_encoded', 
                 'exam_reason_encoded', 'exam_result_encoded', 'exam_type_encoded',
                 'gender_encoded', 'office_municiplaity_encoded', 'region_encoded']]

In [24]:
edf

,test_id,exam_class,year,exam_month_encoded,exam_reason_encoded,exam_result_encoded,exam_type_encoded,gender_encoded,office_municiplaity_encoded,region_encoded
0,0,6,2017,5,6,0,0,2,100,0
1,1,6,2017,5,6,0,0,2,100,0
2,2,6,2017,5,6,1,0,0,74,4
3,3,6,2017,5,6,1,0,2,100,0
4,4,6,2017,5,6,1,0,2,100,0
...,...,...,...,...,...,...,...,...,...,...
1417529,1417529,5,2018,6,0,1,1,0,60,4
1417530,1417530,5,2018,6,0,1,1,0,60,4
1417531,1417531,5,2018,6,0,1,1,0,60,4
1417532,1417532,5,2018,6,0,1,1,0,60,4


In [25]:
##### Step 6.
##### Train the Model with our selected question, given a dataset to predict whether the dataset is going to pass or 
##### fail? In order to predict wether a set of data will pass or fail, we have to set our target predictive dataset 
##### as Y, and our inputdatasets as all other numerical columns.
##### We can use Multiple classifiers and predicter modules to solve this problem from sklearn.
##### We are going to use the RandomForest classifier to train our predictive model, and use a random dataset we made
##### up to test the effectiveness of the model.

In [26]:
edf['year'].unique()

array(['2017', '2022 (up to June 30)', '2018', '2019', '2020', '2021',
       2020, 2018, 2021, 2019, 2017], dtype=object)

In [27]:
###** We've found a string space and numerical repeat problem when we're trying to fit our randomForest model, thus
###** we came back the fix the data in the dataframes using replace.


edf['year'] = edf['year'].replace('2022 (up to June 30)', 2022)
edf['year'] = edf['year'].replace('2017', 2017)
edf['year'] = edf['year'].replace('2018', 2018)
edf['year'] = edf['year'].replace('2019', 2019)
edf['year'] = edf['year'].replace('2020', 2020)
edf['year'] = edf['year'].replace('2021', 2021)

In [28]:
edf['year'].unique()

array([2017, 2022, 2018, 2019, 2020, 2021])

In [29]:
X = edf.loc[:, ['exam_class', 'year', 'exam_month_encoded', 'exam_reason_encoded','exam_type_encoded',
                'gender_encoded', 'office_municiplaity_encoded', 'region_encoded']].values

y = edf.loc[:, ['exam_result_encoded']].values

In [30]:
X

array([[   6, 2017,    5, ...,    2,  100,    0],
       [   6, 2017,    5, ...,    2,  100,    0],
       [   6, 2017,    5, ...,    0,   74,    4],
       ...,
       [   5, 2018,    6, ...,    0,   60,    4],
       [   5, 2018,    6, ...,    0,   60,    4],
       [   5, 2018,    6, ...,    0,   60,    4]])

In [31]:
y

array([[0],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [32]:
type(X)

numpy.ndarray

In [33]:
type(y)

numpy.ndarray

In [34]:
#### The first thing to do after we've created the dataset is to split the dataset to training sets and test sets.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [35]:
print(X_train.shape)
print(X_test.shape)

(1134027, 8)
(283507, 8)


In [36]:
print(y_train.shape)
print(y_test.shape)

(1134027, 1)
(283507, 1)


In [52]:
y_train = y_train.ravel()

In [55]:
y_test = y_test.ravel()

In [59]:
y_train.shape

(1134027,)

In [56]:
y_test.shape

(283507,)

In [ ]:
#### Find the best N estimator for RandomForest by using the 5Fold cross-validation.

In [57]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

lowest_cross_val_error = np.inf
best_n_estimators = None

indices = range(len(X_train))
kf = KFold(n_splits=5, shuffle=True, random_state=0) #将train data再次分为五个fold

for k in range(5,101,5):
    
    errors = []
    
    for train_indices, val_indices in kf.split(indices):
        
        classifier = RandomForestClassifier(n_estimators=k, random_state=0)
        
        classifier.fit(X_train[train_indices],
                       y_train[train_indices])
        
        predicted_val_labels = classifier.predict(X_train[val_indices])
        
        error = np.mean(predicted_val_labels != y_train[val_indices])
        
        errors.append(error)
        
    cross_val_error = np.mean(errors)
    print('n_estimators:', k, 'cross validation error:', cross_val_error)

    if cross_val_error < lowest_cross_val_error:
        lowest_cross_val_error = cross_val_error
        best_n_estimators = k

print('\nBest n_estimators:', best_n_estimators, '\ncross validation error:', lowest_cross_val_error)

n_estimators: 5 cross validation error: 0.39411407528682074
n_estimators: 10 cross validation error: 0.3931590719035613
n_estimators: 15 cross validation error: 0.3925541476912063
n_estimators: 20 cross validation error: 0.39249330254561604
n_estimators: 25 cross validation error: 0.39243069407762576
n_estimators: 30 cross validation error: 0.3923751395912942
n_estimators: 35 cross validation error: 0.3922896036553372
n_estimators: 40 cross validation error: 0.39224727636341195
n_estimators: 45 cross validation error: 0.3921705590573112
n_estimators: 50 cross validation error: 0.39224374942312623
n_estimators: 55 cross validation error: 0.39217849539417243
n_estimators: 60 cross validation error: 0.3921705590767511
n_estimators: 65 cross validation error: 0.3921264681280604
n_estimators: 70 cross validation error: 0.392137049913134
n_estimators: 75 cross validation error: 0.39198008734199885
n_estimators: 80 cross validation error: 0.392052395904703
n_estimators: 85 cross validation er

In [ ]:
## Best n_estimator = 75.
## We will create our randomForest method to train and predict results for our research question.